In [53]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import os
import time
from sklearn import datasets
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
from keras.datasets import cifar10
from keras.utils import to_categorical
import numpy as np

In [54]:
# load train and test dataset
def load_dataset():
    # load dataset
    data_x = np.load('data_x_aumentated_normalized.npy')
    data_y = np.load('data_y_aumentated_normalized.npy')
    #data_x = np.load('data_x_original_augmented.npy')
    #data_y = np.load('data_y_original_augmented.npy')
    #percentage of the dataset which will be on TRAIN - TEST - VALIDATION
    TOTAL_IMAGES = len(data_y)
    print("TOTAL_IMAGES = ", TOTAL_IMAGES)
    
    TRAIN_PERCENTAGE = 0.8
    TEST_PERCENTAGE = 0.1
    VALIDATION_PERCENTAGE = 0.1

    train_size = int(TRAIN_PERCENTAGE * TOTAL_IMAGES)
    test_size = int(TEST_PERCENTAGE * TOTAL_IMAGES)
    validation_size = int(VALIDATION_PERCENTAGE * TOTAL_IMAGES)

    train_x = data_x[:train_size] #taking train images from original data
    train_y = data_y[:train_size]
    data_x = data_x[train_size:] #removing train images from original data
    data_y = data_y[train_size:]

    test_x = data_x[:test_size] #taking test images from original data
    test_y = data_y[:test_size]
    data_x = data_x[test_size:] #removing test images from original data
    data_y = data_y[test_size:]

    validation_x = data_x #all the remainder images we can put on validation
    validation_y = data_y

    del data_x #free some unused memory
    del data_y

    print("train_size = ", len(train_y))
    print("test_size = ", len(test_y))
    print("validation_size = ", len(validation_y))
 # one hot encode target values
    train_y = to_categorical(train_y)
    test_y = to_categorical(test_y)
    validation_y = to_categorical(validation_y)
    return train_x, train_y, test_x, test_y, validation_x, validation_y

In [55]:
# scale pixels
def prep_pixels(train, test, validation):
    # convert from integers to floats
    train_norm = train.astype('float32')
    test_norm = test.astype('float32')
    validation_norm = validation.astype('float32')
    # normalize to range 0-1
    train_norm = train_norm / 255.0
    test_norm = test_norm / 255.0
    validation_norm = validation_norm / 255.0
    # return normalized images
    return train_norm, test_norm, validation_norm

In [56]:
trainX, trainY, testX, testY, validationX, validationY = load_dataset()
#trainX, testX, validationX = prep_pixels(trainX, testX, validationX)
#testX = trainX
#testY = trainY

TOTAL_IMAGES =  2368
train_size =  1894
test_size =  236
validation_size =  238


In [57]:
trainX.shape

(1894, 128, 128, 3)

In [58]:
trainX[0]

array([[[0.56470588, 0.51764706, 0.65490196],
        [0.63137255, 0.56470588, 0.69411765],
        [0.65490196, 0.59607843, 0.70980392],
        ...,
        [0.23529412, 0.18823529, 0.41568627],
        [0.23529412, 0.18823529, 0.43529412],
        [0.29411765, 0.25098039, 0.47058824]],

       [[0.54117647, 0.46666667, 0.64313725],
        [0.57647059, 0.49803922, 0.65098039],
        [0.62352941, 0.54901961, 0.68235294],
        ...,
        [0.21176471, 0.18431373, 0.41568627],
        [0.21176471, 0.17254902, 0.41960784],
        [0.25490196, 0.21176471, 0.45882353]],

       [[0.47058824, 0.39215686, 0.56862745],
        [0.47843137, 0.39215686, 0.57647059],
        [0.54509804, 0.4627451 , 0.61568627],
        ...,
        [0.29411765, 0.24313725, 0.50196078],
        [0.29803922, 0.27058824, 0.51372549],
        [0.33333333, 0.27843137, 0.5254902 ]],

       ...,

       [[0.55686275, 0.4745098 , 0.63137255],
        [0.57647059, 0.4745098 , 0.63921569],
        [0.58823529, 0

In [59]:
#trainX, testX, validationX = prep_pixels(trainX, testX, validationX)

In [60]:
clf1 = tf.keras.models.load_model("displasys_mobilenet-baseline-holdout-100epochs_dataAugmenting_l2_dropout.npy")
#clf1 = tf.keras.models.load_model("cifar-vgg-baseline-holdout-100epochs_all_transfer.npy")

In [61]:
print(type(clf1))

<class 'keras.engine.sequential.Sequential'>


In [62]:
clf2 = tf.keras.models.load_model("displasys_resnet50v2-baseline-holdout-100epochs_dataAugmenting_l2_dropout.npy")
#clf2 = tf.keras.models.load_model("cifar-mobileNet-baseline-holdout-100epochs_all_transfer.npy")

In [63]:
clf3 = tf.keras.models.load_model("resnet50-baseline-holdout-100epochs_normalized.npy")
#clf3 = tf.keras.models.load_model("cifar-resnet50-baseline-holdout-100epochs_all_transfer.npy")

In [64]:
class emsembleModel:
    def __init__(self, classifiers, numClasses):
        self.classifiers = classifiers
        self.numClasses = numClasses
        
    def predict_by_voting(self, dataX):
        preds = []
        for clf in self.classifiers:
            preds.append(clf.predict(dataX))
        
        resultPred = []
        M = len(self.classifiers)
        for i in range(len(dataX)):
            counting = [0 for _ in range(self.numClasses)]
            current_best_pred = preds[0][i]
            for j in range(M):
                pred = preds[j][i]
                predictedClass = np.argmax(pred)
                counting[predictedClass]+=1
                if counting[predictedClass] == max(counting):
                    current_best_pred = pred
            resultPred.append(current_best_pred)
        
        return resultPred
    
    def evaluate_by_voting(self, dataX, dataY):
        pred = self.predict_by_voting(dataX)
        ok = 0
        for i in range(len(pred)):
            expectedClass = np.argmax(dataY[i])
            predictedClass = np.argmax(pred[i])
            if predictedClass == expectedClass:
                ok += 1
        return ok/len(dataY)

    def predict_by_softmax(self, dataX):
        preds = []
        for clf in self.classifiers:
            preds.append(clf.predict(dataX))
        print(preds[0].shape, preds[1].shape, preds[2].shape)
        resultsPred = []
        M = len(self.classifiers)
        for i in range(len(dataX)):
            probs = []
            for j in range(M):
                pred = preds[j][i]
                probs.append(pred)
            compressed_probs = np.array(probs).sum(axis=0)
            compressed_probs = compressed_probs/M
            resultsPred.append(compressed_probs)
        return resultsPred
    
    def evaluate_by_softmax(self, dataX, dataY):
        pred = self.predict_by_softmax(dataX)
        ok = 0
        for i in range(len(pred)):
            expectedClass =  np.argmax(dataY[i])
            predictedClass = np.argmax(pred[i])
            if predictedClass == expectedClass:
                ok += 1
        return ok/len(dataY)
    

In [65]:
classifiers = [clf1, clf2, clf3]

In [66]:
emsemble = emsembleModel(classifiers, 4)

In [67]:
acc = emsemble.evaluate_by_softmax(testX, testY)

8/8 [==============================] - 2s 125ms/step
(236, 4) (236, 4) (236, 4)


In [68]:
print('Accuracy Emsemble using Softmax: %.3f' % (acc * 100.0))

Accuracy Emsemble using Softmax: 94.492


In [69]:
acc = emsemble.evaluate_by_voting(testX, testY)

8/8 [==============================] - 1s 91ms/step


In [70]:
print('Accuracy Emsemble using Voting: %.3f' % (acc * 100.0))

Accuracy Emsemble using Voting: 96.186
